In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
data = pd.read_csv('spam.csv',encoding='latin-1')

data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":'label', "v2":'text'})
print(data.head())
tags = data["label"]
texts = data["text"]

  label                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


### To do: develop an accurate simple neural network model for spam classification (no LSTM, CNN, etc.)

In [2]:
data.isnull().sum()

label    0
text     0
dtype: int64

In [3]:
data.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [4]:
tags = tags.map( {'spam': 1, 'ham': 0} ).astype(int)

In [5]:
X= data['text']
y=tags

In [6]:
X

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: text, Length: 5572, dtype: object

In [7]:
y

0       0
1       0
2       1
3       0
4       0
       ..
5567    1
5568    0
5569    0
5570    0
5571    0
Name: label, Length: 5572, dtype: int32

In [8]:
texts_len=[len(data.loc[i]['text'].split()) for i in range(0, len(data))]

In [9]:
#computing for the total words and average in the dataset
Words_dataset= np.sum(texts_len) 
Words_average= int(np.mean(texts_len))
print("The total words in the dataset is:",Words_dataset,"words")
print("Average words:",Words_average)

The total words in the dataset is: 86335 words
Average words: 15


In [10]:
import sklearn
from sklearn.model_selection import train_test_split
X=np.asanyarray(texts)
y=np.asanyarray(y)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y, test_size=0.2,)

In [11]:
import tensorflow as t
from tensorflow import keras
from tensorflow.keras import layers
texts_new =layers.TextVectorization(max_tokens=Words_dataset,standardize='lower_and_strip_punctuation',output_mode='int',
output_sequence_length=Words_average)
texts_new.adapt(X_train)

In [12]:
embedding_layer = layers.Embedding(input_dim=Words_dataset,output_dim=128,embeddings_initializer='uniform',
input_length=Words_average)

In [13]:
print("The number of datapoints: ",X.shape,X_train.shape,X_test.shape)

The number of datapoints:  (5572,) (4457,) (1115,)


In [14]:
#prepare and define the model
InputL=layers.Input(shape=(1,), dtype=t.string)
VectorL=texts_new(InputL)
EmbedL=embedding_layer(VectorL)
Global_Ave=layers.GlobalAveragePooling1D()(EmbedL)
Flatten_L=layers.Flatten()(Global_Ave)
DenseL=layers.Dense(6, activation='relu')(Flatten_L)
OutputL=layers.Dense(1, activation='sigmoid')(DenseL)
model = keras.Model(InputL, OutputL)

In [15]:
#Compiling the keras model
model.compile(optimizer=keras.optimizers.Adam(),loss=keras.losses.BinaryCrossentropy(label_smoothing=0.5),metrics=['accuracy'])

In [16]:
#Training the model
history=model.fit(X_train,y_train, epochs=100,batch_size=510, validation_data=(X_test, y_test),validation_steps=int(0.2 * len(X_test)))

Epoch 1/100
9/9 [==============================] - 4s 321ms/step - loss: 0.6871 - accuracy: 0.8535 - val_loss: 0.6799 - val_accuracy: 0.8816
Epoch 2/100
9/9 [==============================] - 1s 113ms/step - loss: 0.6735 - accuracy: 0.8961
Epoch 3/100
9/9 [==============================] - 1s 120ms/step - loss: 0.6579 - accuracy: 0.9165
Epoch 4/100
9/9 [==============================] - 1s 116ms/step - loss: 0.6404 - accuracy: 0.9282
Epoch 5/100
9/9 [==============================] - 1s 111ms/step - loss: 0.6220 - accuracy: 0.9396
Epoch 6/100
9/9 [==============================] - 1s 110ms/step - loss: 0.6046 - accuracy: 0.9515
Epoch 7/100
9/9 [==============================] - 1s 111ms/step - loss: 0.5907 - accuracy: 0.9605
Epoch 8/100
9/9 [==============================] - 1s 111ms/step - loss: 0.5819 - accuracy: 0.9650
Epoch 9/100
9/9 [==============================] - 1s 111ms/step - loss: 0.5771 - accuracy: 0.9740
Epoch 10/100
9/9 [==============================] - 1s 111ms/step -

9/9 [==============================] - 1s 126ms/step - loss: 0.5625 - accuracy: 0.9998
Epoch 80/100
9/9 [==============================] - 1s 109ms/step - loss: 0.5625 - accuracy: 0.9998
Epoch 81/100
9/9 [==============================] - 1s 112ms/step - loss: 0.5625 - accuracy: 0.9998
Epoch 82/100
9/9 [==============================] - 1s 114ms/step - loss: 0.5625 - accuracy: 0.9998
Epoch 83/100
9/9 [==============================] - 1s 113ms/step - loss: 0.5625 - accuracy: 0.9998
Epoch 84/100
9/9 [==============================] - 1s 113ms/step - loss: 0.5625 - accuracy: 0.9998
Epoch 85/100
9/9 [==============================] - 1s 113ms/step - loss: 0.5625 - accuracy: 0.9998
Epoch 86/100
9/9 [==============================] - 1s 111ms/step - loss: 0.5625 - accuracy: 0.9998
Epoch 87/100
9/9 [==============================] - 1s 118ms/step - loss: 0.5625 - accuracy: 0.9998
Epoch 88/100
9/9 [==============================] - 1s 121ms/step - loss: 0.5625 - accuracy: 0.9998
Epoch 89/100


In [17]:
#For the Testing Accuracy
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy {:.4f}".format(accuracy))

Testing Accuracy 0.9776


In [18]:
#Loss
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Loss {:.4f}".format(loss))

Testing Loss 0.5724


In [19]:
predictions = (model.predict(X) > 0.5).astype(int)

175/175 [==============================] - 0s 2ms/step


In [20]:
#Predicting teh output csv
spam_output=pd.read_csv("output_spam.csv") 
y_pred = model.predict(spam_output["text"])
y_pred

1/1 [==============================] - 0s 147ms/step


array([[0.23032804],
       [0.2523236 ],
       [0.25337777],
       [0.24396652],
       [0.25269306],
       [0.2908933 ],
       [0.25894946],
       [0.24868798],
       [0.24992892],
       [0.249208  ],
       [0.7500807 ],
       [0.24820088],
       [0.24703826],
       [0.76228964],
       [0.25095218],
       [0.25144485],
       [0.26805943],
       [0.25102705],
       [0.24925983],
       [0.2485277 ]], dtype=float32)